# Class 12 (25.10.2021)

# SQLite
* Serverless transactional SQL Databases.
* Widely used SQL database engine.

## When to avoid SQLite?
* When there are many client programs sending SQL requests to the same database over the network, a client server databse would be better than SQLite.
* High volume websites (database has to be write intensive).
* Very large scale databases. (SQLite is limited to 140TB)

In [1]:
import sqlite3

# Connecting to Database
connection = sqlite3.connect("College.db") # If database is not existing, it will be created and stored on hard disk
cursor = connection.cursor() # Cursor object is used to perform actions on the database (SQL statements)

In [2]:
# Creating tables
#connection.execute("create table tab1(name char(20), age integer)") # This also works
cursor.execute("create table tab1(name char(20), age integer)")
cursor.execute("create table tab2(name char(20), age integer)")

In [3]:
# Inserting values
cursor.execute("insert into tab1 values('xyz', 21)")
cursor.execute("insert into tab2 values('abc', 12)")

# Using placeholders
info = ('efg', 54)
cursor.execute("insert into tab1(name, age) values(?, ?)", info)

In [5]:
# Displaying the table
tab1 = cursor.execute("select * from tab1")
for i in tab1: print(i)

print("-------------------")

tab2 = cursor.execute("select * from tab2")
for i in tab2: print(i)

('xyz', 21)
('efg', 54)
-------------------
('abc', 12)


In [6]:
connection.close() # Closing the connection to the database

## Creating the Database in RAM

In [9]:
import sqlite3

# Each time a DB is created in RAM, it occupies a new region of memory. Hence one can create multiple independent DBs in RAM
connection =  sqlite3.connect(":memory:") # DB is created in RAM instead of HDD
cursor = connection.cursor()
cursor.execute("create table people(name, age)")

# QMark Style of insertion
who = 'Abc'
age = 21
cursor.execute("insert into people values(?,?)", (who, age))

# Named Style
cursor.execute("select * from people where name = :who and age = :age", {"who":who, "age":age})
print(cursor.fetchone())

connection.close()

('Abc', 21)


## Commit

In [10]:
import sqlite3
connection = sqlite3.connect("Student.db")

In [11]:
connection.execute('''
    create table Company (
        ID int primary key not null,
        Name text not null,
        Age int not null,
        Address char(50),
        Salary real
    );
''')

In [14]:
connection.execute("insert into Company (ID, Name, Age, Address, Salary)\
                    values(1, 'ABC', 21, 'Bengaluru 560050', 1200000)"
)

connection.execute("insert into Company (ID, Name, Age, Address, Salary)\
                    values(2, 'DEF', 22, 'Bengaluru 560040', 1100000)"
)

connection.execute("insert into Company (ID, Name, Age, Address, Salary)\
                    values(3, 'GHI', 20, 'Bengaluru 560051', 1250000)"
)
connection.commit()
print("Records inserted successfully")

Records inserted successfully


In [18]:
items = connection.execute("select * from Company")
columns = ("ID", "Name", "Age", "Address", "Salary")
for item in items: print(dict(zip(columns, item)))

{'ID': 1, 'Name': 'ABC', 'Age': 21, 'Address': 'Bengaluru 560050', 'Salary': 1200000.0}
{'ID': 2, 'Name': 'DEF', 'Age': 22, 'Address': 'Bengaluru 560040', 'Salary': 1100000.0}
{'ID': 3, 'Name': 'GHI', 'Age': 20, 'Address': 'Bengaluru 560051', 'Salary': 1250000.0}


In [20]:
connection.execute("update Company set Salary = 3000000 where ID = 3")

items = connection.execute("select * from Company")
columns = ("ID", "Name", "Age", "Address", "Salary")
for item in items: print(dict(zip(columns, item)))

{'ID': 1, 'Name': 'ABC', 'Age': 21, 'Address': 'Bengaluru 560050', 'Salary': 1200000.0}
{'ID': 2, 'Name': 'DEF', 'Age': 22, 'Address': 'Bengaluru 560040', 'Salary': 1100000.0}
{'ID': 3, 'Name': 'GHI', 'Age': 20, 'Address': 'Bengaluru 560051', 'Salary': 3000000.0}


In [21]:
connection.close()

# Class 13 (8.11.2021)

# Fetching all rows of a query result
* **cursor.fetchall()** <br/>
  Fetches all rows of a query result.<br/>
  It returns all rows as list of tuples.<br/>
  An empty list will be returned if there are no records to fetch.<br/><br/>

* **cursor.fetchmany(size)** <br/>
  Returns the number of rows specified by the size argument.<br/>
  It fetches the next set of rows when called repeatedly.<br/>
  Default value for size is 1.<br/>
  This also returns a list of tuples (an empty list will be returned if there are no records to fetch).<br/><br/>

* **cursor.fetchone()** <br/>
  Returns a single row that matches the condition. (None is returned if there are no rows to fetch).


In [35]:
import sqlite3
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()

In [36]:
# Only one statement can be executed at a time. Hence we need to write two connection.execute() statements
connection.execute('''
    create table supplier_groups(
        group_id int primary key,
        group_name text not null
    );
''')

connection.execute('''
    create table suppliers(
        supplier_id int primary key,
        supplier_name text not null,
        group_id int not null,
        foreign key(group_id) references supplier_groups(group_id)
    );
''')

In [37]:
connection.execute('''
    insert into supplier_groups values (1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg');
''')

In [38]:
ret = connection.execute("select * from supplier_groups")
for group in ret:
    print(group)

(1, 'abc')
(2, 'abc')
(3, 'xyz')
(4, 'efg')


In [39]:
cursor.execute("select * from supplier_groups")
p = cursor.fetchmany() # Default size is one
print(p)

[(1, 'abc')]


In [40]:
cursor.execute("select * from supplier_groups")
p1 = cursor.fetchmany(2)
p2 = cursor.fetchmany(2)
print(p1)
print(p2)

[(1, 'abc'), (2, 'abc')]
[(3, 'xyz'), (4, 'efg')]


In [41]:
cursor.execute("select * from supplier_groups")
all = cursor.fetchall()
print(all)

[(1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg')]


In [42]:
cursor.execute("select * from supplier_groups")
one = cursor.fetchone()
print(one)

(1, 'abc')


In [43]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchall())

[(1, 'abc'), (2, 'abc'), (3, 'xyz'), (4, 'efg')]


In [44]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchone())

(1, 'abc')


In [45]:
ret = connection.execute("select * from supplier_groups")
print(ret.fetchmany(2))
print(ret.fetchmany(2))

[(1, 'abc'), (2, 'abc')]
[(3, 'xyz'), (4, 'efg')]


In [46]:
connection.close()

# SQLAlchemy (A powerful Open Source, Cross Platform SQL toolkit written in Python)
* This is a powerful database access toolkit for Python.
* It has Object Relational Mapper (ORM).
* ORM sits between SQLite database and Python program and transforms the dataflow between the database engine and Python object.
* SQLAlchemy allows us to think in terms of objects, i.e provides an OOPS abstraction to SQL databases.
+ ORM is a programming technique for converting the data between incompatible type systems in Object Oriented Programming Languages.
+ The type system used in OOP languages like Python is Non Scalar, i.e, they cannot be expressed in terms of primitive datatypes like int, float, char, etc.
+ OOP languages therefore have to convert Non Scalar data into Scalar in order to interact with the backend data.
+ ORM maps a class to a table. This saves us from writing tedious database interface codes.

In [47]:
import sqlalchemy
print(sqlalchemy.__version__)

1.3.12


# Engine Class
* Connects a *Pool* and *Dialect* together. 
* This provides a source for database connectivity and behaviour.
* An object of Engine class is created using the *create_engine* function.

In [58]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer

# SQL Expression language constructs its expressions against table columns
engine = create_engine('sqlite:///College.db', echo = True) # echo makes the output (SQL statements) verbose

# MetaData is a collection of table objects and their associated schema constructs.
meta = MetaData() # Contains info about the tables and associated objects like index, view, triggers, etc.

# Creating a table
students = Table(
    'Students', meta,
    Column('ID', Integer, primary_key = True),
    Column('Name', String),
    Column('LastName', String)
)

meta.create_all(engine) # Creates the table
connection = engine.connect()

2021-11-11 19:55:02,696 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-11-11 19:55:02,697 INFO sqlalchemy.engine.base.Engine ()
2021-11-11 19:55:02,700 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-11-11 19:55:02,701 INFO sqlalchemy.engine.base.Engine ()
2021-11-11 19:55:02,703 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Students")
2021-11-11 19:55:02,704 INFO sqlalchemy.engine.base.Engine ()
2021-11-11 19:55:02,706 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Students")
2021-11-11 19:55:02,706 INFO sqlalchemy.engine.base.Engine ()
2021-11-11 19:55:02,708 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Students" (
	"ID" INTEGER NOT NULL, 
	"Name" VARCHAR, 
	"LastName" VARCHAR, 
	PRIMARY KEY ("ID")
)


2021-11-11 19:55:02,709 INFO sqlalchemy.engine.base.Engine ()
2021-11-11 19:55:03,657 INFO sqlalchemy.engine.base.Engine COMMIT


# Class 14 (10.11.2021)

In [59]:
connection.execute(students.insert(), [
    {'ID': 1, 'Name' : 'Abc', 'LastName' : 'Efg'},
    {'ID': 2, 'Name' : 'Def', 'LastName' : 'Ghi'}
])

2021-11-11 19:55:09,061 INFO sqlalchemy.engine.base.Engine INSERT INTO "Students" ("ID", "Name", "LastName") VALUES (?, ?, ?)
2021-11-11 19:55:09,062 INFO sqlalchemy.engine.base.Engine ((1, 'Abc', 'Efg'), (2, 'Def', 'Ghi'))
2021-11-11 19:55:09,065 INFO sqlalchemy.engine.base.Engine COMMIT


In [61]:
result = connection.execute(students.select())
print(f'result = {result}')
for row in result: print(row)

2021-11-11 19:58:47,339 INFO sqlalchemy.engine.base.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName" 
FROM "Students"
2021-11-11 19:58:47,341 INFO sqlalchemy.engine.base.Engine ()
result = <sqlalchemy.engine.result.ResultProxy object at 0x7f085bdd7fd0>
(1, 'Abc', 'Efg')
(2, 'Def', 'Ghi')


In [67]:
# Select with condition
filtered = connection.execute(students.select().where(students.c.ID > 1))
for row in filtered: print(row)

2021-11-11 20:25:54,371 INFO sqlalchemy.engine.base.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName" 
FROM "Students" 
WHERE "Students"."ID" > ?
2021-11-11 20:25:54,374 INFO sqlalchemy.engine.base.Engine (1,)
(2, 'Def', 'Ghi')


In [69]:
# Update
connection.execute(students.update().where(students.c.Name == 'Abc').values(Name = 'Xyz'))
result = connection.execute(students.select())
print(f'result = {result}')
for row in result: print(row)

2021-11-11 20:28:03,950 INFO sqlalchemy.engine.base.Engine UPDATE "Students" SET "Name"=? WHERE "Students"."Name" = ?
2021-11-11 20:28:03,954 INFO sqlalchemy.engine.base.Engine ('Xyz', 'Abc')
2021-11-11 20:28:03,959 INFO sqlalchemy.engine.base.Engine COMMIT
2021-11-11 20:28:05,851 INFO sqlalchemy.engine.base.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName" 
FROM "Students"
2021-11-11 20:28:05,853 INFO sqlalchemy.engine.base.Engine ()
result = <sqlalchemy.engine.result.ResultProxy object at 0x7f0843018160>
(1, 'Xyz', 'Efg')
(2, 'Def', 'Ghi')


In [70]:
# Delete
connection.execute(students.delete().where(students.c.LastName == 'Ghi'))
result = connection.execute(students.select())
print(f'result = {result}')
for row in result: print(row)

2021-11-11 20:30:43,669 INFO sqlalchemy.engine.base.Engine DELETE FROM "Students" WHERE "Students"."LastName" = ?
2021-11-11 20:30:43,671 INFO sqlalchemy.engine.base.Engine ('Ghi',)
2021-11-11 20:30:43,673 INFO sqlalchemy.engine.base.Engine COMMIT
2021-11-11 20:30:43,826 INFO sqlalchemy.engine.base.Engine SELECT "Students"."ID", "Students"."Name", "Students"."LastName" 
FROM "Students"
2021-11-11 20:30:43,828 INFO sqlalchemy.engine.base.Engine ()
result = <sqlalchemy.engine.result.ResultProxy object at 0x7f0842cb1d30>
(1, 'Xyz', 'Efg')


In [71]:
connection.close()